## Introduction to Transact-SQL

***

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

import pyodbc

%matplotlib inline

pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.option_context('float_format','{:.2f}'.format)

random.seed(0)
np.random.seed(0)
np.set_printoptions(suppress=True)

## Load Data from SQL database

### MSSQL

In [2]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-ARQLULI\SQLEXPRESS2019;'
                      'Database=AdventureWorksLT2012;'
                      'Trusted_Connection=yes;')

In [3]:
#cursor = conn.cursor()

In [4]:
#cursor

In [5]:
#cursor.execute('SELECT * FROM SALESLT.Address')

## Challenge 1: Retrieve Customer Data

In [6]:
pd.read_sql_query("SELECT TOP 5 * FROM SALESLT.Customer", conn)

,CustomerID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,CompanyName,SalesPerson,EmailAddress,Phone,PasswordHash,PasswordSalt,rowguid,ModifiedDate
0,1,False,Mr.,Orlando,N.,Gee,None,A Bike Store,adventure-works\pamela0,orlando0@adventure-works.com,245-555-0173,L/Rlwxzp4w7RWmEgXX+/A7cXaePEPcp+KwQhl2fJL7w=,1KjXYs4=,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2001-08-01
1,2,False,Mr.,Keith,None,Harris,None,Progressive Sports,adventure-works\david8,keith0@adventure-works.com,170-555-0127,YPdtRdvqeAhj6wyxEsFdshBDNXxkCXn+CRgbvJItknw=,fs1ZGhY=,E552F657-A9AF-4A7D-A645-C429D6E02491,2002-08-01
2,3,False,Ms.,Donna,F.,Carreras,None,Advanced Bike Components,adventure-works\jillian0,donna0@adventure-works.com,279-555-0130,LNoK27abGQo48gGue3EBV/UrlYSToV0/s87dCRV7uJk=,YTNH5Rw=,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2001-09-01
3,4,False,Ms.,Janet,M.,Gates,None,Modular Cycle Systems,adventure-works\jillian0,janet1@adventure-works.com,710-555-0173,ElzTpSNbUW1Ut+L5cWlfR7MF6nBZia8WpmGaQPjLOJA=,nm7D5e4=,FF862851-1DAA-4044-BE7C-3E85583C054D,2002-07-01
4,5,False,Mr.,Lucy,None,Harrington,None,Metropolitan Sports Supply,adventure-works\shu0,lucy0@adventure-works.com,828-555-0186,KJqV15wsX3PG8TS5GSddp6LFFVdd3CoRftZM/tP0+R4=,cNFKU4w=,83905BDC-6F5E-4F71-B162-C98DA069F38A,2002-09-01


In [7]:
pd.read_sql_query("SELECT TOP 5 Title, FirstName, MiddleName, LastName, Suffix \
                   FROM SALESLT.Customer", conn)

,Title,FirstName,MiddleName,LastName,Suffix
0,Mr.,Orlando,N.,Gee,None
1,Mr.,Keith,None,Harris,None
2,Ms.,Donna,F.,Carreras,None
3,Ms.,Janet,M.,Gates,None
4,Mr.,Lucy,None,Harrington,None


In [8]:
pd.read_sql_query("SELECT TOP 5 SalesPerson, Title + ' ' + LastName AS CustomerName, Phone\
                   FROM SALESLT.Customer", conn)

,SalesPerson,CustomerName,Phone
0,adventure-works\pamela0,Mr. Gee,245-555-0173
1,adventure-works\david8,Mr. Harris,170-555-0127
2,adventure-works\jillian0,Ms. Carreras,279-555-0130
3,adventure-works\jillian0,Ms. Gates,710-555-0173
4,adventure-works\shu0,Mr. Harrington,828-555-0186


## Challenge 2: Retrieve Customer and Sales Data

In [9]:
pd.read_sql_query("SELECT CAST(CustomerID AS VARCHAR) + ': ' + CompanyName AS CustomerCompany\
                   FROM SALESLT.Customer",conn)

,CustomerCompany
0,1: A Bike Store
1,2: Progressive Sports
2,3: Advanced Bike Components
3,4: Modular Cycle Systems
4,5: Metropolitan Sports Supply
...,...
842,30113: Nearby Cycle Shop
843,30115: Retreat Inn
844,30116: Technical Parts Manufacturing
845,30117: Totes & Baskets Company


In [10]:
pd.read_sql_query("SELECT TOP 5 SalesOrderNumber + ' (' + STR(RevisionNumber, 1) + ')' AS OrderRevision,  \
                   CONVERT(NVARCHAR(30), OrderDate, 102) AS OrderDate \
                   FROM SalesLT.SalesOrderHeader;", conn)

,OrderRevision,OrderDate
0,SO71774 (1),2004.06.01
1,SO71776 (1),2004.06.01
2,SO71780 (1),2004.06.01
3,SO71782 (1),2004.06.01
4,SO71783 (1),2004.06.01


## Challenge 3: Retrieve Customer Contact Details

In [11]:
pd.read_sql_query("SELECT TOP 5 FirstName +' '+ ISNULL (MiddleName + ' ', '') + LastName AS CustomerName\
                  FROM SalesLT.Customer", conn)

,CustomerName
0,Orlando N. Gee
1,Keith Harris
2,Donna F. Carreras
3,Janet M. Gates
4,Lucy Harrington


In [12]:
pd.read_sql_query("SELECT CustomerID, COALESCE(EmailAddress, Phone) AS PrimaryContact \
                   FROM SalesLT.Customer;", conn)

,CustomerID,PrimaryContact
0,1,orlando0@adventure-works.com
1,2,keith0@adventure-works.com
2,3,donna0@adventure-works.com
3,4,janet1@adventure-works.com
4,5,lucy0@adventure-works.com
...,...,...
842,30113,raja0@adventure-works.com
843,30115,dora0@adventure-works.com
844,30116,wanda0@adventure-works.com
845,30117,robert13@adventure-works.com


In [13]:
pd.read_sql_query("SELECT TOP 5 SalesOrderID, OrderDate, \
                   CASE \
                       WHEN ShipDate IS NULL THEN 'Awaiting Shipment'\
                       ELSE 'Shipped'\
                   END AS ShippingStatus\
                   FROM SalesLT.SalesOrderHeader;", conn)

,SalesOrderID,OrderDate,ShippingStatus
0,71774,2004-06-01,Shipped
1,71776,2004-06-01,Shipped
2,71780,2004-06-01,Shipped
3,71782,2004-06-01,Shipped
4,71783,2004-06-01,Shipped


In [14]:
conn.close()

#### Python code done by Dennis Lam